In [4]:
from pyrep import PyRep
from pyrep.objects import Dummy
from pyrep.robots.robot_component import RobotComponent
import time
import pickle

In [5]:
class StarkitRobot(RobotComponent):
    def __init__(self, joint_names):
        super().__init__(0, "Telo_Surrogat", joint_names=joint_names)

In [6]:
class PyRepEnvironment(object):
    def __init__(self, scene_file, joint_names, imu_name):
        self.scene_file = scene_file
        self.pr = PyRep()
        self.pr.launch(self.scene_file, headless=False)
        self.pr.start()
        self.robot = StarkitRobot(joint_names)
        self.imu_object = Dummy(imu_name)
        
    def __del__(self):
        self.pr.stop()
        self.pr.shutdown()

    def reset(self):
        self.joint_positions = self.robot.get_joint_positions()
        quat = self.imu_object.get_quaternion()
        return self.joint_positions, quat
        

    def step(self, action):
        if action:
            self.joint_positions = action
        
        self.robot.set_joint_target_positions(self.joint_positions)
        self.pr.step()
        quat = self.imu_object.get_quaternion()
        return (self.joint_positions, quat), 0

# Эксперименты

## Basic cappelia

In [7]:
from basic_walk.utils import BaseAgent, DirectEnvironment
import sys

In [8]:
active_joints = ['Leg_right_10', 'Leg_right_9', 'Leg_right_8', 'Leg_right_7', 'Leg_right_6', 'Leg_right_5',
                 'hand_right_4',
                 'hand_right_3', 'hand_right_2', 'hand_right_1', 'Tors1', 'Leg_left_10', 'Leg_left_9',
                 'Leg_left_8',
                 'Leg_left_7', 'Leg_left_6', 'Leg_left_5', 'hand_left_4', 'hand_left_3', 'hand_left_2',
                 'hand_left_1']

In [43]:
env = DirectEnvironment(active_joints, "Dummy_H", "/home/alex/CoppeliaSim/programming/remoteApiBindings/python/python/")
# env = PyRepEnvironment('scenes/basic_scene.ttt', active_joints, "Dummy_H")
agent = BaseAgent(active_joints)

base_agent_actions = []
try:
    state = env.reset()
    #     print(state[0])
    while True:
        action = agent.act(state)
        base_agent_actions.append(action)
#         print(state[1])
        state, _ = env.step(action)
#         time.sleep(0.05)
except:
    print("Unexpected error:", sys.exc_info())
    del agent
    del env

Unexpected error: (<class 'KeyboardInterrupt'>, KeyboardInterrupt(), <traceback object at 0x7f4ebbfcb100>)


In [22]:
base_agent_actions = []

In [6]:
base_agent_actions = base_agent_actions if 'base_agent_actions' in globals() else []

if len(base_agent_actions):
    with open('base_agent_actions.pickle', 'wb') as f:
        pickle.dump(base_agent_actions, f)
else:
    with open('base_agent_actions.pickle', 'rb') as f:
        base_agent_actions = pickle.load(f)

len(base_agent_actions)

819

# PyRep Slow replay

In [13]:
SCENE_FILE = 'scenes/basic_scene.ttt'
pr = PyRep()
pr.launch(SCENE_FILE, headless=False)
pr.start()
robot = StarkitRobot(active_joints)
robot.set_control_loop_enabled(True)

In [14]:
for action in base_agent_actions:
    if not action:
        break
    robot.set_joint_target_positions(action)
    pr.step()
#     time.sleep(0.1)

In [8]:
base_agent_actions[0]

[-8.600714000011478e-05,
 0.4034568855840074,
 -0.7958604497213635,
 0.3924035641373561,
 -8.600714000011478e-05,
 0,
 1.745,
 0.0,
 0.0,
 0.524,
 0.0,
 8.600714000011478e-05,
 -0.4034568855840074,
 0.7958604497213635,
 -0.3924035641373561,
 8.600714000011478e-05,
 0,
 -1.745,
 0.0,
 0.0,
 -0.524]

In [10]:
action = base_agent_actions[0][:-4]
action += [0., 0., 0.] + base_agent_actions[0][-1:]
robot.set_joint_positions(action)
pr.step()

time.sleep(10)

action = base_agent_actions[0][:-3]
action += [0., 0.] + base_agent_actions[0][-1:]
robot.set_joint_positions(action)
pr.step()

In [ ]:
base_agent_actions

In [37]:
action = base_agent_actions[0][:-4]
action += [0., 0., 0.] + base_agent_actions[0][-1:]
steps = 300
for i in range(steps):
    action[-4] = base_agent_actions[0][-4] * (steps - i) / steps
    robot.set_joint_positions(action)
    pr.step()

In [35]:
action

[-8.600714000011478e-05,
 0.4034568855840074,
 -0.7958604497213635,
 0.3924035641373561,
 -8.600714000011478e-05,
 0,
 1.745,
 0.0,
 0.0,
 0.524,
 0.0,
 8.600714000011478e-05,
 -0.4034568855840074,
 0.7958604497213635,
 -0.3924035641373561,
 8.600714000011478e-05,
 0,
 -0.043625000000000004,
 0.0,
 0.0,
 -0.524]

In [36]:
robot.get_joint_positions()

[-0.0029219721909612417,
 0.4025570750236511,
 -0.7978057265281677,
 0.3903423845767975,
 0.00015797215746715665,
 0.00032267256756313145,
 1.7436635494232178,
 9.884436440188438e-05,
 3.563112113624811e-05,
 0.5239410996437073,
 -2.9750033718300983e-05,
 0.003080591093748808,
 -0.4023972153663635,
 0.7974095344543457,
 -0.3895416259765625,
 -0.001226728199981153,
 0.0014231501845642924,
 -1.7432948350906372,
 -8.482335397275165e-05,
 -3.0443714422290213e-05,
 -0.5239300727844238]

In [15]:
for i in range(50):
    robot.set_joint_positions(base_agent_actions[0])
    pr.step()
    time.sleep(0.1)

In [15]:
pr.stop()
pr.shutdown()

## PyREP only hand

In [7]:
SCENE_FILE = 'scenes/basic_scene.ttt'
pr = PyRep()
pr.launch(SCENE_FILE, headless=False)
pr.start()
robot = StarkitRobot(["hand_left_4", ])

In [15]:
robot.get_joint_positions()

[-1.7435201406478882]

In [12]:
robot.set_joint_positions([-1.7, ], disable_dynamics=True)
pr.step()

time.sleep(3)

robot.set_joint_positions([-1.6, ], disable_dynamics=True)
pr.step()

time.sleep(3)

robot.set_joint_positions([-1.5, ], disable_dynamics=True)
pr.step()

time.sleep(3)

robot.set_joint_positions([-1.4, ], disable_dynamics=True)
pr.step()

In [8]:
robot.set_joint_target_positions([-1.7, ])
pr.step()

time.sleep(3)

robot.set_joint_target_positions([-1.6, ])
pr.step()

time.sleep(3)

robot.set_joint_target_positions([-1.5, ])
pr.step()

time.sleep(3)

robot.set_joint_target_positions([-1.4, ])
pr.step()

In [9]:
robot.get_joint_positions()

[-1.5437815189361572]

In [10]:
pr.stop()
pr.shutdown()

# PyREP replay

In [ ]:
env = PyRepEnvironment('scenes/basic_scene.ttt', active_joints, "Dummy_H")

try:
    state = env.reset()
    for action in base_agent_actions:
        state, _ = env.step(action)
    while True:
        state, _ = env.step(base_agent_actions[-1])
except:
    print("Unexpected error:", sys.exc_info())
    del env

## PyREP

In [ ]:
# env = DirectEnvironment(active_joints, "Dummy_H", "/home/alex/CoppeliaSim/programming/remoteApiBindings/python/python/")
env = PyRepEnvironment('scenes/basic_scene.ttt', active_joints, "Dummy_H")
agent = BaseAgent(active_joints)


try:
    state = env.reset()
    #     print(state[0])
    while True:
        action = agent.act(state)
        base_agent_actions.append(action)
#         print(state[1])
        state, _ = env.step(action)
#         time.sleep(0.05)
except:
    print("Unexpected error:", sys.exc_info())
    del agent
    del env